In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

from tetris import Tetris, rewards
from tetris_master import TetrisMaster2
from Agent import DQNAgent

In [2]:
w, h = 10, 20
game_len = 50
epoch = 10000
betta = 0.3
gamma = 0.89
rf = 4
lr = 1e-5

In [3]:
env = Tetris(w, h)

# Initialize the agent
state_dim = (env.board.shape, 1)
action_dim = w, 4
batch_size = 1

model = TetrisMaster2(bw=w, bh=h).double()

agent = DQNAgent(model, state_dim, action_dim)

In [6]:
# Training the DQN agent
episodes = 1000
it = tqdm(range(episodes))
for episode in it:
    state = env.clear_board()
    total_reward = 0
    done = False
    while not done:
        action = agent.select_action(state)
        next_state, reward, done = env.step(action)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward
    print('Done')
    loss = agent.replay(batch_size)
    it.set_postfix(str=f"Episode: {episode + 1}, loss {round(loss, rf)}, Total Reward: {round(total_reward, rf)}")
it.close()

# Evaluate the trained agent
test_episodes = 10
it = tqdm(range(test_episodes))
for _ in it:
    state = env.clear_board()
    total_reward = 0
    done = False
    while not done:
        action = agent.select_action(state)
        next_state, reward, done = env.step(action)
        state = next_state
        total_reward += reward
    it.set_postfix(str=f"Test Episode, loss {round(loss, rf)}, Total Reward: {round(total_reward, rf)}")

try:
    torch.save({
        'model_state_dict': agent.model.state_dict(),
    }, 'checkpoints/test_model'.format(epoch, best_loss))
except KeyboardInterrupt:
    torch.save({
        'model_state_dict': agent.model.state_dict(),
    }, 'checkpoints/test_model'.format(epoch, best_loss))

it.close()

  0%|▎                                                                | 4/1000 [00:00<00:30, 32.34it/s, str=Episode: 7, loss 18.6632, Total Reward: -0.0202]

Done
Done
Done
Done
Done
Done
Done


  1%|▌                                                               | 8/1000 [00:00<00:28, 34.61it/s, str=Episode: 11, loss 14.1125, Total Reward: -0.0202]

Done
Done
Done
Done


  1%|▋                                                              | 11/1000 [00:14<21:52,  1.33s/it, str=Episode: 11, loss 14.1125, Total Reward: -0.0202]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses)
# plt.show()